## Experiments with D$^\alpha$ seeding for $k$-means++

### Imports

In [ ]:
import numpy as np
import scipy as sp
import itertools
import math
import matplotlib.pyplot as plt

### Function Definitions

In [ ]:
def sampleMixdiffGauss(k,d,n,MU,I,sigma_list): #Sample from a balanced mixture of k-Gaussians with given mean and covariance of the form sigma^2I
    X=np.zeros((n,d))
    for i in range(n):
        gauss_no=np.random.randint(k)
        X[i,:]=np.random.multivariate_normal(MU[gauss_no,:],sigma_list[gauss_no]*I)
    return X       

In [ ]:
def samplenonUnifMixdiffGauss(k,d,n,MU,I,sigma_list,prob_list): #Sample from a unbalanced mixture of k-Gaussians with given mean and covariance of the form sigma^2I
    X=np.zeros((n,d))
    for i in range(n):
        #gauss_no=np.random.randint(k)
        #probabilities = [0.2, 0.5, 0.3]
        list_of_dist= list(range(k))
        gauss_no=np.random.choice(list_of_dist, 1, p=prob_list)
        X[i,:]=np.random.multivariate_normal(MU[gauss_no[0],:],sigma_list[gauss_no[0]]*I)
    return X  

In [ ]:
def sampleMixStudentt(k,d,n,MU,I,sigma_list,df): #Sample from a balanced mixture of k-Student-t Distributions with given mean and covariance of the form sigma^2I
    X=np.zeros((n,d))
    for i in range(n):
        gauss_no=np.random.randint(k)
        Z=sp.stats.multivariate_t(MU[gauss_no,:],sigma_list[gauss_no]*I,df)
        X[i,:]=Z.rvs()
    return X 

In [ ]:
def choosekCenters(X,alpha,k,d): #alpha >=2, seeding step
    T=np.zeros((k,d)) #To store centers
    nprime = np.size(X,0)
    n=np.shape(X)[0]
    Imd_Dist_center=np.zeros((k,nprime))
    center0=np.random.randint(n)
    T[0,:]=X[center0,:]
    Imd_Dist_center=np.zeros((k,nprime)) #Stores the best cost row by row
    alpha_cost_0=(np.linalg.norm(X-T[0,:],axis=1))**(alpha/2)
    Imd_Dist_center[0,:]=alpha_cost_0
    for j in range(1,k):
        #Imd_Prob_center=np.amin(Imd_Dist_center,axis=0)
        rand_vals =np.random.uniform(size=1) * np.sum(Imd_Dist_center[j-1,:])
        random_center_idx = np.searchsorted(np.cumsum(Imd_Dist_center[j-1,:]), rand_vals)
        random_center=X[random_center_idx[0],:]
        T[j,:]=random_center
        new_alpha_cost=np.zeros((j+1,nprime))
        for l in range(j+1):
          new_alpha_cost[l,:]=(np.linalg.norm(X-T[l,:],axis=1))**(alpha/2)
        Imd_Dist_center[j,:]=np.amin(new_alpha_cost,axis=0)
    return T

In [ ]:
def assignclusterlabels(X,T,k,d):
    nprime=np.size(X,0)
    Labels=np.zeros(nprime)
    Imd_Dist_center=np.zeros((k,nprime))
    for j in range(k):
        Imd_Dist_center[j,:]=(np.linalg.norm(X-T[j,:],axis=1))    
    Labels=np.argmin(Imd_Dist_center, axis=0)
    Min_Dist_center=np.amin(Imd_Dist_center, axis=0)
    return Labels, Min_Dist_center

In [ ]:
def Lloydsuntilconv(X,C,Min_Dist_Center,k,MaxIter,d,epsilon): #C is labels after seeding, MaxIter is number of iterations to run the Llyods step
    Lab=C
    flag = 0
    i=0
    nprime=np.shape(X)[0]
    Old_Min_Dist_Center=np.sum(Min_Dist_Center)
    while(i < MaxIter and flag==0):
        
        Centroid=np.zeros((k,d))
        Imd_Dist_center=np.zeros((k,nprime))
        for j in range(k):
            temp_arr = X[Lab==j]
            temp_centroid= np.mean(temp_arr, axis=0)
            Centroid[j,:]=temp_centroid
            Imd_Dist_center[j,:]=(np.linalg.norm(X-temp_centroid,axis=1)) 
        NewLabels=assignclusterlabels(X,Centroid,k,d)[0]
        New_Min_Dist_center=np.amin(Imd_Dist_center, axis=0)
        Lab=NewLabels
        if(np.abs(Old_Min_Dist_Center - np.sum(New_Min_Dist_center)) <= epsilon):
            flag=1
        else:
            Old_Min_Dist_Center=np.sum(New_Min_Dist_center)
        i+=1        
    return NewLabels, New_Min_Dist_center, i

### Global Variables

In [ ]:
MaxIter =10000 #For no of Lloyds iterations
epsilon=0.001 #For Lloyds convergence tolerance.
alpha_list=np.arange(2,42,4) #list of alpha_values
npoints=1000 #Total no of points to be generated from the distributions
N=5000 #No of times to run the seeding algorithm. The average performance over these N iterations is then reported.
DELTA= 50 #Separation between two clusters

### Instances

1) $I_1$: A mixture of $8$ unbalanced Gaussians with the centers/means of the Gaussians placed on the corners of a cube with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=3$).\
2) $I_2$: A mixture of $4$ unbalanced Gaussians with the centers/means of the Gaussians placed on the corners of a square with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=2$).\
1) $I_3$: A mixture of $4$ balanced Gaussians with the centers/means of the Gaussians placed on the corners of a square with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=2$).\
2) $I_4$: A mixture of $4$ balanced Gaussians with the centers/means of the Gaussians placed on the corners of a square with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=2$), except one that has a covariance $\sigma^2I$, with $\sigma^2=400$.
5) $I_5$: A mixture of $8$ balanced Gaussians with the centers/means of the Gaussians placed on the corners of a cube with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=3$).\
6) $I_6$: A mixture of $8$ balanced Gaussians with the centers/means of the Gaussians placed on the corners of a cube with edge length of $2\Delta=100$ (default). All the covariances are identity matrices (in $d=3$), except one that has a covariance $\sigma^2I$, with $\sigma^2=800$.\
7) $I_{7a},\ldots I_{7d}$: A mixture of $4$ balanced bivariate student-t distributions (see Figure \ref{fig:studenttpdf}), with different degrees of freedom, $\nu = \{1.6,2,5,10\}$, where the location parameter is centered on the corners of a square with edge length of $2\Delta=100$ (default).

Note 1: In all cases, we select the centers using $D^{\alpha}$-sampling for different $\alpha >= 2$. However, we do not run any additional steps of Llyods. \
Note 2: The k-means cost of the clustering is stored in the variable kMeansCost_I*_BL and kMeansCost_I*_AL, to represent the k-means cost of a clustering on an instance $I_*$ before Lloyds and after Lloyds respectively.\
Note 3: Increasing the value of $\Delta$ or $\alpha$ by a large amount may lead to overflow errors.

### Run $k$-means ++ on $I_1$

In [ ]:
### Instances
k = 8 #No of clusters
d=3 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA,DELTA]
MU[1,:]=[-DELTA,DELTA,-DELTA]
MU[2,:]=[DELTA,DELTA,DELTA]
MU[3,:]=[DELTA,DELTA,-DELTA]
MU[4,:]=[DELTA,-DELTA,DELTA]
MU[5,:]=[DELTA,-DELTA,-DELTA]
MU[6,:]=[-DELTA,-DELTA,DELTA]
MU[7,:]=[-DELTA,-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1,1,1,1,1]
prob_list=[0.125,0.125,0.125,0.125,0.125,0.125,0.24,0.01]

In [ ]:
XI1=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I1_BL=np.zeros((A,N))
kMeansCost_I1_AL=np.zeros((A,N))
TimetoConv_I1=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI1,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI1,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI1,T,k,d)[0]
        FinLabels, Min_dist_center, TimetoConv_I1[a,m]= Lloydsuntilconv(XI1,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I1_AL[a,m]=np.sum(Min_dist_center)
        kMeansCost_I1_BL[a,m]=np.sum(Old_Min_dist_center)

In [ ]:
#Find the average cost 
kMeansCost_I1_avg_AL=np.nanmean(kMeansCost_I1_AL,axis=1)
kMeansCost_I1_avg_BL=np.nanmean(kMeansCost_I1_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I1_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I1withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()



In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I1_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I1woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

### Run $k$-means ++ on $I_2$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
prob_list=[0.25,0.25,0.49,0.01]

In [ ]:
XI2=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I2_BL=np.zeros((A,N))
kMeansCost_I2_AL=np.zeros((A,N))
TimetoConv_I2=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI2,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI2,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI2,T,k,d)[0]
        FinLabels, Min_dist_center, TimetoConv_I2[a,m]= Lloydsuntilconv(XI2,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I2_AL[a,m]=np.sum(Min_dist_center) 
        kMeansCost_I2_BL[a,m]=np.sum(Old_Min_dist_center) 

In [ ]:
#Find the average cost 
kMeansCost_I2_avg_AL=np.nanmean(kMeansCost_I2_AL,axis=1)
kMeansCost_I2_avg_BL=np.nanmean(kMeansCost_I2_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I2_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I2_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I2withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()




In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I2_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I2woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

### Run $k$-means ++ on $I_3$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
prob_list=[0.25,0.25,0.25,0.25]

In [ ]:
XI3=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I3_BL=np.zeros((A,N))
kMeansCost_I3_AL=np.zeros((A,N))
TimetoConv_I3=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI3,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI3,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI3,T,k,d)[0]
        kMeansCost_I3_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I3[a,m]= Lloydsuntilconv(XI3,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I3_AL[a,m]=np.sum(Min_dist_center) 

In [ ]:
#Find the average cost 
kMeansCost_I3_avg_AL=np.nanmean(kMeansCost_I3_AL,axis=1)
kMeansCost_I3_avg_BL=np.nanmean(kMeansCost_I3_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I3_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I3_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I3withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()




In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I3_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I3woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()


In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[400,1,1,1]
prob_list=[0.25,0.25,0.25,0.25]

In [ ]:
XI4=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I4_BL=np.zeros((A,N))
kMeansCost_I4_AL=np.zeros((A,N))
TimetoConv_I4=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI4,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI4,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI4,T,k,d)[0]
        kMeansCost_I4_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I4[a,m]= Lloydsuntilconv(XI4,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I4_AL[a,m]=np.sum(Min_dist_center) 

In [ ]:
#Find the average cost 
kMeansCost_I4_avg_AL=np.nanmean(kMeansCost_I4_AL,axis=1)
kMeansCost_I4_avg_BL=np.nanmean(kMeansCost_I4_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I4_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I4_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I4withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()






In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I4_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I4woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

### $I_5$

In [ ]:
### Instances
k = 8 #No of clusters
d=3 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA,DELTA]
MU[1,:]=[-DELTA,DELTA,-DELTA]
MU[2,:]=[DELTA,DELTA,DELTA]
MU[3,:]=[DELTA,DELTA,-DELTA]
MU[4,:]=[DELTA,-DELTA,DELTA]
MU[5,:]=[DELTA,-DELTA,-DELTA]
MU[6,:]=[-DELTA,-DELTA,DELTA]
MU[7,:]=[-DELTA,-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1,1,1,1,1]
prob_list=[0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125]

In [ ]:
XI5=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I5_BL=np.zeros((A,N))
kMeansCost_I5_AL=np.zeros((A,N))
TimetoConv_I5=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI5,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI5,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI5,T,k,d)[0]
        kMeansCost_I5_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I5[a,m]= Lloydsuntilconv(XI5,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I5_AL[a,m]=np.sum(Min_dist_center) 

In [ ]:
#Find the average cost 
kMeansCost_I5_avg_AL=np.nanmean(kMeansCost_I5_AL,axis=1)
kMeansCost_I5_avg_BL=np.nanmean(kMeansCost_I5_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I5_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I5_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I5withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()





In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I5_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I5woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

### $I_6$

In [ ]:
### Instances
k = 8 #No of clusters
d=3 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA,DELTA]
MU[1,:]=[-DELTA,DELTA,-DELTA]
MU[2,:]=[DELTA,DELTA,DELTA]
MU[3,:]=[DELTA,DELTA,-DELTA]
MU[4,:]=[DELTA,-DELTA,DELTA]
MU[5,:]=[DELTA,-DELTA,-DELTA]
MU[6,:]=[-DELTA,-DELTA,DELTA]
MU[7,:]=[-DELTA,-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[800,1,1,1,1,1,1,1]
prob_list=[0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125]

In [ ]:
XI6=samplenonUnifMixdiffGauss(k,d,npoints,MU,I,sigma_squared_list,prob_list)

In [ ]:
#Here we calculate the k-means cost as a function of alpha.
A=len(alpha_list)
kMeansCost_I6_BL=np.zeros((A,N))
kMeansCost_I6_AL=np.zeros((A,N))
TimetoConv_I6=np.zeros((A,N))
for a in range(A):
    for m in range(N):
        T= choosekCenters(XI6,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI6,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI6,T,k,d)[0]
        kMeansCost_I6_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I6[a,m]= Lloydsuntilconv(XI6,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I6_AL[a,m]=np.sum(Min_dist_center) 

In [ ]:
#Find the average cost 
kMeansCost_I6_avg_AL=np.nanmean(kMeansCost_I6_AL,axis=1)
kMeansCost_I6_avg_BL=np.nanmean(kMeansCost_I6_BL,axis=1)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I6_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
ax.plot(alpha_list, kMeansCost_I6_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I6withandwoLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()





In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I6_avg_BL, label="Without Lloyds", c='b',linewidth=2.0, linestyle='solid')
#ax.plot(alpha_list, kMeansCost_I1_avg_AL, label="With Lloyds", c='b',linewidth=2.0, linestyle='-.')

plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
fig.savefig('I6woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

### $I_7$

#### $I_{7a}$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
df=1.6

XI7a=sampleMixStudentt(k,d,npoints,MU,I,sigma_squared_list,df)

#Here we calculate the k-means cost as a function of alpha.


A=len(alpha_list)
kMeansCost_I7a_BL=np.zeros((A,N))
kMeansCost_I7a_AL=np.zeros((A,N))
TimetoConv_I7a=np.zeros((A,N))

for a in range(A):
    for m in range(N):
        T= choosekCenters(XI7a,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI7a,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI7a,T,k,d)[0]
        kMeansCost_I7a_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I7a[a,m]= Lloydsuntilconv(XI7a,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I7a_AL[a,m]=np.sum(Min_dist_center) 
        
#Find the average cost 
kMeansCost_I7a_avg_AL=np.nanmean(kMeansCost_I7a_AL,axis=1)
kMeansCost_I7a_avg_BL=np.nanmean(kMeansCost_I7a_BL,axis=1)
TimetoConv_I7a_avg=np.nanmean(TimetoConv_I7a,axis=1)


#### $I_{7b}$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
df=2

XI7b=sampleMixStudentt(k,d,npoints,MU,I,sigma_squared_list,df)

#Here we calculate the k-means cost as a function of alpha.


A=len(alpha_list)
kMeansCost_I7b_BL=np.zeros((A,N))
kMeansCost_I7b_AL=np.zeros((A,N))
TimetoConv_I7b=np.zeros((A,N))

for a in range(A):
    for m in range(N):
        T= choosekCenters(XI7b,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI7b,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI7b,T,k,d)[0]
        kMeansCost_I7b_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I7b[a,m]= Lloydsuntilconv(XI7b,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I7b_AL[a,m]=np.sum(Min_dist_center) 
        
#Find the average cost 
kMeansCost_I7b_avg_AL=np.nanmean(kMeansCost_I7b_AL,axis=1)
kMeansCost_I7b_avg_BL=np.nanmean(kMeansCost_I7b_BL,axis=1)
TimetoConv_I7b_avg=np.nanmean(TimetoConv_I7b,axis=1)


#### $I_{7c}$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
df=5

XI7c=sampleMixStudentt(k,d,npoints,MU,I,sigma_squared_list,df)

#Here we calculate the k-means cost as a function of alpha.


A=len(alpha_list)
kMeansCost_I7c_BL=np.zeros((A,N))
kMeansCost_I7c_AL=np.zeros((A,N))
TimetoConv_I7c=np.zeros((A,N))

for a in range(A):
    for m in range(N):
        T= choosekCenters(XI7c,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI7c,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI7c,T,k,d)[0]
        kMeansCost_I7c_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I7c[a,m]= Lloydsuntilconv(XI7c,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I7c_AL[a,m]=np.sum(Min_dist_center) 
        
#Find the average cost 
kMeansCost_I7c_avg_AL=np.nanmean(kMeansCost_I7c_AL,axis=1)
kMeansCost_I7c_avg_BL=np.nanmean(kMeansCost_I7c_BL,axis=1)
TimetoConv_I7c_avg=np.nanmean(TimetoConv_I7c,axis=1)


#### $I_{7d}$

In [ ]:
k = 4 #No of clusters
d=2 #Dimension where the points live
MU=np.zeros((k,d))
MU[0,:]=[-DELTA,DELTA]
MU[1,:]=[DELTA,DELTA]
MU[2,:]=[DELTA,-DELTA]
MU[3,:]=[-DELTA,-DELTA]
I=np.identity(d)
sigma_squared_list=[1,1,1,1]
df=10

XI7d=sampleMixStudentt(k,d,npoints,MU,I,sigma_squared_list,df)

#Here we calculate the k-means cost as a function of alpha.

A=len(alpha_list)
kMeansCost_I7d_BL=np.zeros((A,N))
kMeansCost_I7d_AL=np.zeros((A,N))
TimetoConv_I7d=np.zeros((A,N))

for a in range(A):
    for m in range(N):
        T= choosekCenters(XI7d,alpha_list[a],k,d)
        Old_Min_dist_center=assignclusterlabels(XI7d,T,k,d)[1]
        Old_Lab=assignclusterlabels(XI7d,T,k,d)[0]
        kMeansCost_I7d_BL[a,m]=np.sum(Old_Min_dist_center) 
        FinLabels, Min_dist_center, TimetoConv_I7d[a,m]= Lloydsuntilconv(XI7d,Old_Lab,Old_Min_dist_center,k,MaxIter,d,epsilon)
        kMeansCost_I7d_AL[a,m]=np.sum(Min_dist_center) 
        
#Find the average cost 
kMeansCost_I7d_avg_AL=np.nanmean(kMeansCost_I7d_AL,axis=1)
kMeansCost_I7d_avg_BL=np.nanmean(kMeansCost_I7d_BL,axis=1)
TimetoConv_I7d_avg=np.nanmean(TimetoConv_I7d,axis=1)


### Plot of all Student-t distributions together

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I7a_avg_BL, c='r', label='df=1.6', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7b_avg_BL, c='g', label='df=2', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7c_avg_BL, c='b', label='df=5', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7d_avg_BL, c='k', label='df=10', linewidth=2.0)


plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I7woLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)
ax.plot(alpha_list, kMeansCost_I7a_avg_AL, c='r', label='df=1.6', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7b_avg_AL, c='g', label='df=2', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7c_avg_AL, c='b', label='df=5', linewidth=2.0)
ax.plot(alpha_list, kMeansCost_I7d_avg_AL, c='k', label='df=10', linewidth=2.0)


plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('k-means cost',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I7withLloyds.eps', format='eps',bbox_inches = "tight")
plt.show()

#### Lloyds Convergence time for Student-t distributions

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)

ax.plot(alpha_list, TimetoConv_I7a_avg, c='r', label='df=1.6', linewidth=2.0)
ax.plot(alpha_list, TimetoConv_I7b_avg, c='g', label='df=2', linewidth=2.0)
ax.plot(alpha_list, TimetoConv_I7c_avg, c='b', label='df=5', linewidth=2.0)
ax.plot(alpha_list, TimetoConv_I7d_avg, c='k', label='df=10', linewidth=2.0)


plt.xlabel(r'$\alpha$',fontsize=20)
plt.ylabel('Lloyds Steps',fontsize=20)
plt.xticks(alpha_list)
plt.legend()
fig.savefig('I7timetoconv.eps', format='eps',bbox_inches = "tight")
plt.show()

In [ ]:
Z1=sp.stats.t(df=1.6)
Z2=sp.stats.t(df=2)
Z3=sp.stats.t(df=5)
Z4=sp.stats.t(df=10)
Z5=sp.stats.norm()
x=np.linspace(-4,4,100)

In [ ]:
fig = plt.figure(dpi=100)
ax  = fig.add_subplot(111)

ax.plot(x, Z1.pdf(x), c='r', label='df=1.6', linewidth=2.0)
ax.plot(x, Z2.pdf(x), c='g', label='df=2', linewidth=2.0)
ax.plot(x, Z3.pdf(x), c='b', label='df=5', linewidth=2.0)
ax.plot(x, Z4.pdf(x), c='k', label='df=10', linewidth=2.0)
ax.plot(x, Z5.pdf(x), c='y', label='standard normal', linewidth=2.0)


plt.xlabel('x',fontsize=20)
plt.ylabel('density',fontsize=20)
plt.legend()
fig.savefig('Studenttwithdf.eps', format='eps',bbox_inches = "tight")
plt.show()